In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torchvision.models as models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True

!pip install pyheif
import pyheif

classes = ['Hyundai SOLARIS',
 'KIA Rio',
 'Skoda Octavia',
 'Volkswagen Polo',
 'Volkswagen Tiguan']

classThresholds = np.array([0.60005697, 0.82278588, 0.77659323, 0.74224129, 0.67364123])

def load_image(pathToImage):
  try:
    pilImage = Image.open(pathToImage)
    pilImage = pilImage.convert('RGB')
    return pilImage
  except:
    imRaw = pyheif.read(pathToImage)
    pilImage = Image.frombytes(mode=imRaw.mode, size=imRaw.size, data=imRaw.data)
    pilImage = pilImage.convert('RGB')
    return pilImage


net = nn.Sequential( *list(models.resnet50(pretrained=False).children())[:-1],\
                    nn.Flatten(1),
                    nn.Linear(in_features=2048, out_features=len(classes), bias=True)
                    )

pathToNet=...
net.load_state_dict(torch.load(pathToNet))
net.cuda()
net.eval()

def getImageClassProbabilities(image, shouldUseConfidences=False): 
  image= image.convert('RGB')

  transform = transforms.Compose([transforms.Resize((1024,1024)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
                                ])
  
  out = net(transform(image).unsqueeze(0).cuda())
  probabilities = F.softmax(out, dim=-1)

  result = dict()
  result["probabilities"] = dict()
  for index in range(len(classes)):
    result["probabilities"][classes[index]] = round(probabilities[0][index].item(), 4)

  if shouldUseConfidences:
    confidencesPerClass = torch.sigmoid(out.detach().cpu()).numpy()
    mostProbClass = np.argmax(confidencesPerClass)

    result["confidence"] = bool(confidencesPerClass[0][mostProbClass] >= classThresholds[mostProbClass])

  return result


In [ ]:
import pickle

pathToFeatureExtractor = ...
with open(..., 'rb') as f:
  vectorsInfo = pickle.load(f)


import torchvision.models as models
modelForExtractor = nn.Sequential( *list(models.resnet50(pretrained=False).children())[:-1],\
                      nn.Flatten(1),
                      nn.Linear(in_features=2048, out_features=1556, bias=True)
                     )
modelForExtractor.load_state_dict(torch.load(...))
featureExtractor = nn.Sequential(*list(modelForExtractor.children())[:-1]).cuda()
featureExtractor.eval()


def findCloseAutos(image):
  image= image.convert('RGB')

  transform = transforms.Compose([transforms.Resize((1024,1024)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
                                ])
  
  features = featureExtractor(transform(image).unsqueeze(0).cuda()).detach().cpu().numpy()

  distances = []

  for index, featureVector in enumerate(vectorsInfo["meanFeatureVectors"]):
    distances.append((np.mean((features - featureVector)**2), index))
  distances = sorted(distances)[:10]
  result = [" ".join(vectorsInfo["classes"][distance[1]].split("_")[:-1]) for distance in distances]
  
  return result

In [ ]:
!pip install flask-ngrok

from flask import Flask, jsonify,request
from PIL import Image
from io import BytesIO
import base64
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def hello_world():
  return "hello_world"

@app.route("/recognition", methods=['POST'])
def recognition():
  json_data = request.get_json()
  im = Image.open(BytesIO(base64.b64decode(json_data['content'])))
  result = getImageClassProbabilities(im)
  return jsonify(result)

@app.route("/recognitionExtended", methods=['POST'])
def recognitionExtended():
  json_data = request.get_json()
  im = Image.open(BytesIO(base64.b64decode(json_data['content'])))
  result = getImageClassProbabilities(im, True)
  return jsonify(result)

@app.route("/suggestion", methods=['POST'])
def suggestion():
  json_data = request.get_json()
  im = Image.open(BytesIO(base64.b64decode(json_data['content'])))
  return jsonify(findCloseAutos(im))

if __name__ == '__main__':
  app.run()

In [ ]:
imageRaw = R"..."

In [ ]:
from PIL import Image
import io
import base64
imageRaw = R"..."
im = Image.open(io.BytesIO(base64.b64decode(imageRaw)))

In [ ]:
getImageClassProbabilities(im)